In [ ]:
from datetime import datetime
from  os import name, path
from io import StringIO
from time import sleep
import json

import utils
from pipe import sort
import pprint 
from AvailableBalances import AvailableBalance
from Ledgers import Ledgers
import Orders
import Trades  
import pandas as pd

In [ ]:
av = AvailableBalance()
bal = av.get_avalailable_balances().sort_values('Currency')
balanceResult = bal[bal['Balance']>0.1]
print(balanceResult)


### Get balances together with currency details

In [ ]:
marketData = pd.DataFrame(av.Market)
merged = pd.merge(marketData, balanceResult,left_on='name',right_on='Currency')
merged.drop(columns=["Currency","aclass"])

### Download history trade data
- report_id will be persisted in ./data/report_<ledger|trades>.json
- if report_id already available, download report from API
- if there is no report_id, request report at kraken and wait for being ready for download

In [ ]:
print()
print(f'trades history:')
print("-------------------")
th = Trades.TradeHistory()
dataTrades = th.get_from_Export(startDate=utils.date2nix(datetime(2021,1,1)))
print(dataTrades.head())

### Download history ledger data

In [ ]:
print()
print(f'ledgers history:')
print("-------------------")
ledger = Ledgers()
dataLedger = ledger.get_from_Export(startDate=utils.date2nix(datetime(2021,1,1)))
print(dataLedger.head())

### Filter ledger data for currency from balance data

In [ ]:
if(not dataLedger is None):
    mergedForBalance=pd.merge(dataLedger, balanceResult, left_on="asset",right_on="Currency", how="left").sort_values(by="time")
    mergedForBalance=mergedForBalance[~mergedForBalance["type"].isin( ["rollover","margin"])]
    mergedWithTrades = pd.merge(mergedForBalance,dataTrades[['txid','type','price','cost']],left_on="refid",right_on="txid")
    mergedWithTrades.reindex(mergedForBalance.index)